In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:
soup = BeautifulSoup(source, 'lxml')

In [4]:
#print(soup.prettify())

In [5]:
article = soup.find('h1', class_ = 'firstHeading')
article.text

'List of postal codes of Canada: M'

In [6]:
para = soup.find('div', class_="mw-parser-output")
a = para.table.text
a = a.split('\n')
type(a)

list

In [7]:
char = ''
for i in range(len(a)):
    if char in a:
        a.remove(char)
a.remove('Postcode')
a.remove('Borough')
a.remove('Neighbourhood')


In [8]:
codes = a[0:len(a):3]
borough = a[1:len(a):3]
neighbourhood = a[2:len(a):3]

In [9]:
b = list(range(0,len(a),3))
len(b)


288

In [10]:
df = pd.DataFrame(index = list(range(0,len(b))),columns = ['Postcode','Borough','Neighbourhood'])

In [11]:
for i in range(0,288):
    df['Postcode'][i] = codes[i]
    df['Borough'][i] = borough[i]
    df['Neighbourhood'][i] = neighbourhood[i]

In [12]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [13]:
df = df[df['Borough'] != 'Not assigned']
df = df.reset_index()

In [14]:
df.drop('index', axis = 1, inplace = True)

In [15]:
for i in range(0,len(df)):
    if df['Neighbourhood'][i] == 'Not assigned':
        df['Neighbourhood'][i] = df['Borough'][i]

In [16]:
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [17]:
df1 = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(lambda col: ', '.join(col))
df1

Postcode  Borough    
M1B       Scarborough                                       Rouge, Malvern
M1C       Scarborough               Highland Creek, Rouge Hill, Port Union
M1E       Scarborough                    Guildwood, Morningside, West Hill
M1G       Scarborough                                               Woburn
M1H       Scarborough                                            Cedarbrae
                                               ...                        
M9N       York                                                      Weston
M9P       Etobicoke                                              Westmount
M9R       Etobicoke      Kingsview Village, Martin Grove Gardens, Richv...
M9V       Etobicoke      Albion Gardens, Beaumond Heights, Humbergate, ...
M9W       Etobicoke                                              Northwest
Name: Neighbourhood, Length: 103, dtype: object

In [18]:
df2 = pd.DataFrame(df1)

In [19]:
df2 = df2.reset_index()


In [20]:
df2.rename(columns = {0:'Neighbourhood'})

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [21]:
df2.shape

(103, 3)

In [22]:
coord = pd.read_csv('Geospatial_Coordinates.csv')
coord.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [23]:
coord = coord.rename(columns = {'Postal Code':'Postcode'})
coord.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [24]:
final_df = pd.merge(df2, coord,on ='Postcode')

In [25]:
final_df.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [26]:
import geocoder
from geopy import Nominatim
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [27]:
import folium

In [28]:
ca_map  = folium.Map(location = [43.653963, -79.387207], zoom_start = 10)
ca_map

In [29]:
for lat, long, bor, neigh in zip(final_df['Latitude'],final_df['Longitude'], final_df['Borough'], final_df['Neighbourhood']):
    label = '{},{}'.format(neigh,bor)
    label = folium.Popup(label)
    folium.CircleMarker(
    location = [lat, long],
    popup = label,
    radius = 5,
    colour = 'blue',
    fill = True ,
    fill_colour = 'blue',
    fill_opacity = 0.2).add_to(ca_map)
ca_map

In [30]:
tor_df = final_df[final_df['Borough'].str.contains('Tor')].reset_index()

In [31]:
tor_df
tor_df.drop('index', axis = 1)
tor_df = tor_df.rename(columns = {'Neighbourhood':'Neighborhood'})

In [32]:
tor_map = folium.Map(location = [43.653963, -79.387207], zoom_state = 20)
tor_map

In [33]:
for lat , long , bor, neigh in zip(tor_df['Latitude'], tor_df['Longitude'], tor_df['Borough'], tor_df['Neighborhood']):
    label = '{},{}'.format(neigh,bor)
    label = folium.Popup(label)
    folium.CircleMarker(
    location = [lat, long],
    popup = label,
    radius = 5,
    colour = 'blue',
    fill = True ,
    fill_colour = 'blue',
    fill_opacity = 0.2).add_to(tor_map)
tor_map

<h2>Defining Four Sqaure Credentials</h2>

In [34]:
CLIENT_ID = 'TLLNCF03T1REFMX4ONVH5BDATKSE0CWPKC32OBTHCOZEHUGU' # your Foursquare ID
CLIENT_SECRET = 'XMODX54H2YDL2U5KLBA1TJUDUH2BU4XLQ40MM35DG5JXG2AA' # your Foursquare Secret
VERSION = '20190101' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TLLNCF03T1REFMX4ONVH5BDATKSE0CWPKC32OBTHCOZEHUGU
CLIENT_SECRET:XMODX54H2YDL2U5KLBA1TJUDUH2BU4XLQ40MM35DG5JXG2AA


In [35]:
import requests

In [36]:
import json
from pandas.io.json import json_normalize

In [37]:
tor_df['Neighborhood'].loc[0]

'The Beaches'

<h3>Exploring top 10 places in 'The Beaches'</h3>

In [38]:
LIMIT = 50
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    tor_df['Latitude'].loc[0], 
    tor_df['Longitude'].loc[0], 
    radius, 
    LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?&client_id=TLLNCF03T1REFMX4ONVH5BDATKSE0CWPKC32OBTHCOZEHUGU&client_secret=XMODX54H2YDL2U5KLBA1TJUDUH2BU4XLQ40MM35DG5JXG2AA&v=20190101&ll=43.67635739999999,-79.2930312&radius=500&limit=50'

In [39]:
results = requests.get(url).json()

In [40]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [41]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869


In [42]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Toronto

In [43]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [44]:
tor_venues = getNearbyVenues(names=tor_df['Neighborhood'],
                                   latitudes=tor_df['Latitude'],
                                   longitudes=tor_df['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [45]:
print(tor_venues.shape)
tor_venues.head()

(1165, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [46]:
tor_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",50,50,50,50,50,50
Berczy Park,50,50,50,50,50,50
"Brockton, Exhibition Place, Parkdale Village",22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,20,20,20,20,20,20
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",15,15,15,15,15,15
"Cabbagetown, St. James Town",45,45,45,45,45,45
Central Bay Street,50,50,50,50,50,50
"Chinatown, Grange Park, Kensington Market",50,50,50,50,50,50
Christie,16,16,16,16,16,16


In [47]:
print('There are {} uniques categories.'.format(len(tor_venues['Venue Category'].unique())))

There are 214 uniques categories.


In [48]:
# one hot encoding
tor_onehot = pd.get_dummies(tor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot['Neighborhood'] = tor_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
tor_onehot = tor_onehot[fixed_columns]

tor_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
tor_onehot.shape

(1165, 214)

In [50]:
tor_grouped = tor_onehot.groupby('Neighborhood').mean().reset_index()
tor_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.060000,0.00,...,0.00,0.00,0.00000,0.000000,0.00,0.020000,0.000000,0.0000,0.00,0.00
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00,0.00,0.00000,0.000000,0.00,0.020000,0.000000,0.0000,0.00,0.00
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.0000,0.00,0.00
3,Business Reply Mail Processing Centre 969 Eastern,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.0000,0.00,0.00
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.066667,0.066667,0.066667,0.133333,0.133333,0.066667,0.000000,0.00,...,0.00,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.0000,0.00,0.00
5,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.0000,0.00,0.00
6,Central Bay Street,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.00,...,0.00,0.00,0.00000,0.000000,0.00,0.020000,0.000000,0.0000,0.00,0.00
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00,0.02,0.00000,0.000000,0.00,0.060000,0.000000,0.0600,0.02,0.00
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.0000,0.00,0.00
9,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.02,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.0200,0.00,0.02


In [51]:
tor_grouped.shape

(38, 214)

In [52]:
num_top_venues = 5

for hood in tor_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tor_grouped[tor_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0           Steakhouse  0.08
1                 Café  0.06
2  American Restaurant  0.06
3          Coffee Shop  0.06
4     Asian Restaurant  0.06


----Berczy Park----
                venue  freq
0         Coffee Shop  0.08
1        Cocktail Bar  0.06
2         Cheese Shop  0.04
3  Seafood Restaurant  0.04
4            Beer Bar  0.04


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0  Breakfast Spot  0.09
1     Coffee Shop  0.09
2            Café  0.09
3   Burrito Place  0.05
4  Sandwich Place  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0    Light Rail Station  0.10
1           Yoga Studio  0.05
2         Auto Workshop  0.05
3  Fast Food Restaurant  0.05
4            Smoke Shop  0.05


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0    Airport Lounge  

In [53]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [54]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_grouped['Neighborhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Steakhouse,Coffee Shop,American Restaurant,Asian Restaurant,Café,Gastropub,Pizza Place,Bar,Hotel,Seafood Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Cheese Shop,Beer Bar,Steakhouse,Farmers Market,Café,Bakery,Park
2,"Brockton, Exhibition Place, Parkdale Village",Café,Coffee Shop,Breakfast Spot,Caribbean Restaurant,Sandwich Place,Restaurant,Burrito Place,Bar,Stadium,Bakery
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Park,Skate Park,Brewery,Restaurant,Recording Studio,Burrito Place,Butcher,Comic Shop
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Plane,Coffee Shop,Boat or Ferry,Bar,Sculpture Garden,Harbor / Marina,Boutique,Airport


<h1> Clustering </h1>

In [55]:
from sklearn.cluster import KMeans
kclusters = 5

tor_grouped_clustering = tor_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [56]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
tor_merged = tor_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

tor_merged.head() # check the last columns!

,index,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,37,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Health Food Store,Pub,Wings Joint,Cupcake Shop,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run,Discount Store
1,41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,3,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Dessert Shop,Bakery,Indian Restaurant,Sports Bar,Café
2,42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,3,Sandwich Place,Pizza Place,Gym,Food & Drink Shop,Board Shop,Brewery,Burger Joint,Burrito Place,Fast Food Restaurant,Italian Restaurant
3,43,M4M,East Toronto,Studio District,43.659526,-79.340923,3,Café,Coffee Shop,American Restaurant,Bakery,Italian Restaurant,Gym,Diner,Park,Music Store,Middle Eastern Restaurant
4,44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Park,Swim School,Bus Line,Wings Joint,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run,Discount Store


In [57]:
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighborhood'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examining Clusters


<h3>Cluster 1</h3>

In [58]:
tor_merged.loc[tor_merged['Cluster Labels'] == 0, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,-79.293031,0,Trail,Health Food Store,Pub,Wings Joint,Cupcake Shop,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run,Discount Store
5,M4P,-79.390197,0,Gym,Clothing Store,Park,Breakfast Spot,Sandwich Place,Hotel,Food & Drink Shop,Deli / Bodega,Dumpling Restaurant,Donut Shop
10,M4W,-79.377529,0,Park,Playground,Trail,Building,Wings Joint,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run
23,M5P,-79.411307,0,Trail,Jewelry Store,Mexican Restaurant,Sushi Restaurant,Wings Joint,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run


<h3>Cluster 2</h3>

In [59]:
tor_merged.loc[tor_merged['Cluster Labels'] == 1, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,M5N,-79.416936,1,Pool,Garden,Wings Joint,Cupcake Shop,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run,Discount Store,Diner


<h3>Cluster 3</h3>

In [60]:
tor_merged.loc[tor_merged['Cluster Labels'] == 2, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M4N,-79.38879,2,Park,Swim School,Bus Line,Wings Joint,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run,Discount Store


<h3> Cluster 4 </h3>

In [61]:
tor_merged.loc[tor_merged['Cluster Labels'] == 3, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M4K,-79.352188,3,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Dessert Shop,Bakery,Indian Restaurant,Sports Bar,Café
2,M4L,-79.315572,3,Sandwich Place,Pizza Place,Gym,Food & Drink Shop,Board Shop,Brewery,Burger Joint,Burrito Place,Fast Food Restaurant,Italian Restaurant
3,M4M,-79.340923,3,Café,Coffee Shop,American Restaurant,Bakery,Italian Restaurant,Gym,Diner,Park,Music Store,Middle Eastern Restaurant
6,M4R,-79.405678,3,Coffee Shop,Clothing Store,Sporting Goods Shop,Bagel Shop,Park,Chinese Restaurant,Café,Burger Joint,Rental Car Location,Restaurant
7,M4S,-79.388790,3,Sandwich Place,Dessert Shop,Gym,Italian Restaurant,Café,Pizza Place,Coffee Shop,Sushi Restaurant,Farmers Market,Seafood Restaurant
9,M4V,-79.400049,3,Coffee Shop,Pub,Fried Chicken Joint,Restaurant,Liquor Store,Light Rail Station,Sports Bar,Bagel Shop,Supermarket,Sushi Restaurant
11,M4X,-79.367675,3,Coffee Shop,Bakery,Italian Restaurant,Pizza Place,Restaurant,Pub,Pet Store,Grocery Store,Café,Playground
12,M4Y,-79.383160,3,Coffee Shop,Gym,Burger Joint,Men's Store,Gay Bar,Gastropub,Restaurant,Burrito Place,Café,Ramen Restaurant
13,M5A,-79.360636,3,Coffee Shop,Park,Bakery,Café,Mexican Restaurant,Theater,Breakfast Spot,Pub,Restaurant,Electronics Store
14,M5B,-79.378937,3,Coffee Shop,Café,Middle Eastern Restaurant,Fast Food Restaurant,Clothing Store,Italian Restaurant,Theater,Bookstore,Ramen Restaurant,Beer Bar


<h3>Cluster 5</h3>

In [62]:
tor_merged.loc[tor_merged['Cluster Labels'] == 4, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,M4T,-79.38316,4,Gym,Trail,Playground,Airport Food Court,Airport Gate,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run
